# Predicting Satisfiability of SAT-3 Problems
## Solving the Classification Task using a ***Graphical Neural Network (GNN)***

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
__counter__ = random.randint(0,2e9)

from IPython.display import HTML, display

In [3]:
import os, shutil
import json

from tuning import tune_parameters
from data_loader import dataset_processing
from train import training, testing

Device is: cuda:0


A helper function to ensure that the dataset is made from scratch when NN is tested. 

In [4]:
def delete_folder_contents(folders):
    for folder in folders:
        for filename in os.listdir(folder):
            file_path = os.path.join(folder, filename)
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)

### Test set sampled from the same distribution as the training and validation sets 

In order to train the model, a *training set* is used alongside with a *validation set*. The latter is used to measure the performance of the model. The training and validation set were extracted from the 80% (60%-20%) of the data and the rest 20% was kept to evaluate the model after its training (*test set*).  

Erase the previous dataset created by PyTorch

In [7]:
delete_folder_contents(["./raw", "./processed"])

**Create the dataset**: The dataset is created from the raw data in such a format that it can be loader by the *DataLoader* module of torch.geometric. <br>

The dataset must be provided to Pytorch in a graph format, so in this preprocessing are constructed the *vertices, the edges, the labels* etc.

In [8]:
pos_weight = dataset_processing(separate_test=False)

Start the data processing...

Satisfiable CNFs   : 3701
Unsatisfiable CNFs : 2700

Ratio of SAT   : 0.5782
Ratio of UNSAT : 0.4218

Training set size: 5120
Test set size: 1280

Processing completed.


**Tune parameters**: Tune the parameters of the model (GNN itself) as well as parameters regarding the training process e.g. *batch-size*, *weight decay* etc. More specidically, the parameters that are tuned are the following.
<ol><li>Model parameters:</li>
    <ul> 
        <li>Number of layers</li>
        <li>Dropout rate</li>
        <li>Neurons' density in its last Linear Layers</li>
        <li>Embedding size for the Graph Transformer Operator (TransformerConv)</li>
        <li>Attention heads for the Graph Transformer Operator (TransformerConv)</li><br>
    </ul>
    <li>Training parameters:</li>
    <ul> 
        <li>Batch size used</li>
        <li>Learning rate</li>
        <li>Weight decay</li>
    </ul>
</ol>

The parameters of the final model are the ones that result in the minimum ***validation error***. <br>
Also, ***early stopping*** is used in order to avoid *overfitting*.

In [ ]:
# Tune parameters
best_parameters = tune_parameters(pos_weight=pos_weight)

# Show best parameters
print(f'Best hyperparameters were: {best_parameters}')
# Store best parameters
with open('./best_parameters_same_sets.txt', 'w') as f:
    f.write(json.dumps(best_parameters))

f.close()

**Train with the best parameters**: The GNN is trained using the selected parameters.

In [9]:
# Access the best parameters in order to train final model
with open('./best_parameters_same_sets.txt') as f:
    data = f.read()

best_parameters_loaded = json.loads(data)

print('\nNow training with the best parameters\n')
training(best_parameters_loaded, make_err_logs=True)


Now training with the best parameters

Dataset loading...


Processing...
100%|█████████████████████████████████████████████████████████████████████████████| 5120/5120 [00:07<00:00, 727.86it/s]
Done!


Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 0.5869
Validation Loss : 0.5740

EPOCH | 1
Training Loss   : 0.5366
Validation Loss : 0.5505

EPOCH | 2
Training Loss   : 0.4253
Validation Loss : 0.4146

EPOCH | 3
Training Loss   : 0.2238
Validation Loss : 0.2393

EPOCH | 4
Training Loss   : 0.1586
Validation Loss : 0.3276

EPOCH | 5
Training Loss   : 0.1180
Validation Loss : 1.0890

EPOCH | 6
Training Loss   : 0.0865
Validation Loss : 0.1049

EPOCH | 7
Training Loss   : 0.0732
Validation Loss : 0.2686

EPOCH | 8
Training Loss   : 0.0639
Validation Loss : 0.1308

EPOCH | 9
Training Loss   : 0.0464
Validation Loss : 0.2120

EPOCH | 10
Training Loss   : 0.0412
Validation Loss : 0.0851

EPOCH | 11
Training Loss   : 0.0332
Validation Loss : 0.0170

EPOCH | 12
Training Loss   : 0.0186
Validation Loss : 0.0098

EPOCH | 13
Training Loss   : 0.0143
Validation Loss : 0.0686

EPOCH | 14
Training Loss   : 0.0110
Validation Loss : 0.1842

EPOCH | 15


0.0052026261037099175

The following Figure shows the ***training and validation set errors***, the epoch where the ***early stopping*** was activated, as well as the epoch of the final ***selected model***.

In [1]:
display(HTML('<img src="plots/train_valid_error.png?%d" height=400 width=400>' % __counter__))

NameError: name 'HTML' is not defined

**Test the model**: Predict the satisfiability of the clauses in the *Test Set* and get the corresponding metrics.

In [11]:
print('\nResults on the test set:\n')
testing(params=best_parameters_loaded)


Results on the test set:

Dataset loading...


Processing...
100%|█████████████████████████████████████████████████████████████████████████████| 1280/1280 [00:01<00:00, 724.32it/s]
Done!


Dataset loading completed

Model loading...
Model loading completed


Test set metrics:

 Confusion matrix: 
 [[521   0]
 [  0 759]]
F1 Score  : 1.0000
Accuracy  : 1.0000
Precision : 1.0000
Recall    : 1.0000
ROC AUC   : 1.0000
Test Loss : 0.0036169828461424915


The following **Figures** show:
<ol>
<li>The <b>confusion matrix</b> for the test set-prediction</li>
<li>The <b>ROC-AUC curve</b> for the test set-prediction</li>
<li>The <b>precision recall curve</b> for the test set-prediction</li>
</ol>

In [12]:
print("\n1.")
display(HTML('<img src="plots/cm.png?%d" height=500 width=500>' % __counter__))
print("2.")
display(HTML('<img src="plots/roc_auc.png?%d" height=450 width=450>' % __counter__))
print("3.")
display(HTML('<img src="plots/pr.png?%d" height=450 width=450>' % __counter__))


1.


2.


3.


### Test set sampled from a different distribution from the training and validation sets 

Here, again, in order to train the model, a training set is used alongside with a validation set. However, now, they are both sampled randomly from the smaller 3-SAT problems i.e. have less than 250 variables. The test set consists of 200 problem instances that have 250 variables and 1065 clauses each. Since in this case we would like to predict the satisfiability of the problems that the well known algorithms cannot predict, ***we want to measure the performance of the model for bigger instances that it has not seen during its training***.

The training-tuning process is the same as showcased above.

In [12]:
# Erase the previous dataset created by PyTorch
delete_folder_contents(["./raw", "./processed"])

In [8]:
# Create the dataset
pos_weight = dataset_processing(separate_test=True)

Start the data processing...

Satisfiable CNFs   : 3601
Unsatisfiable CNFs : 2600

Ratio of SAT   : 0.5807
Ratio of UNSAT : 0.4193

Training set size: 6200
Test set size: 200

Processing completed.


In [ ]:
# Tune parameters
best_parameters = tune_parameters(pos_weight=pos_weight)

# Show best parameters
print(f'Best hyperparameters were: {best_parameters}')
# Store best parameters
with open('best_parameters_diff_test.txt', 'w') as f:
    f.write(json.dumps(best_parameters))

f.close()

In [9]:
# Train with the best parameters

# Access the best parameters in order to train final model
with open('best_parameters_diff_test.txt') as f:
    data = f.read()

best_parameters_loaded = json.loads(data)

print('\nNow training with the best parameters\n')
training(best_parameters_loaded, make_err_logs=True)


Now training with the best parameters

Dataset loading...


Processing...
100%|█████████████████████████████████████████████████████████████████████████████| 6200/6200 [00:08<00:00, 754.26it/s]
Done!


Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 0.5707
Validation Loss : 0.6077

EPOCH | 1
Training Loss   : 0.4637
Validation Loss : 1.2736

EPOCH | 2
Training Loss   : 0.3474
Validation Loss : 0.3346

EPOCH | 3
Training Loss   : 0.2451
Validation Loss : 0.1784

EPOCH | 4
Training Loss   : 0.1733
Validation Loss : 0.3987

EPOCH | 5
Training Loss   : 0.1499
Validation Loss : 0.1641

EPOCH | 6
Training Loss   : 0.1296
Validation Loss : 0.2813

EPOCH | 7
Training Loss   : 0.1241
Validation Loss : 0.2997

EPOCH | 8
Training Loss   : 0.1294
Validation Loss : 0.1490

EPOCH | 9
Training Loss   : 0.1064
Validation Loss : 0.1292

EPOCH | 10
Training Loss   : 0.1062
Validation Loss : 0.0969

EPOCH | 11
Training Loss   : 0.1005
Validation Loss : 0.1055

EPOCH | 12
Training Loss   : 0.0953
Validation Loss : 0.1565

EPOCH | 13
Training Loss   : 0.0927
Validation Loss : 0.1870

EPOCH | 14
Training Loss   : 0.0879
Validation Loss : 0.2328

EPOCH | 15


0.10546847768366718

In [10]:
# Test the model
print('\nResults on the test set:\n')
testing(params=best_parameters_loaded)


Results on the test set:

Dataset loading...


Processing...
100%|███████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 423.13it/s]
Done!


Dataset loading completed

Model loading...
Model loading completed


Test set metrics:

 Confusion matrix: 
 [[  0   0]
 [100 100]]
F1 Score  : 0.6667
Accuracy  : 0.5000
Precision : 0.5000
Recall    : 1.0000
ROC AUC   : 0.5000
Test Loss : 2.3636052792093585


The following **Figures** show:
<ol>
<li>The <b>confusion matrix</b> for the test set-prediction</li>
<li>The <b>ROC-AUC curve</b> for the test set-prediction</li>
<li>The <b>precision recall curve</b> for the test set-prediction</li>
</ol>

In [11]:
print("\n1.")
display(HTML('<img src="plots/cm.png?%d" height=500 width=500>' % __counter__))
print("2.")
display(HTML('<img src="plots/roc_auc.png?%d" height=450 width=450>' % __counter__))
print("3.")
display(HTML('<img src="plots/pr.png?%d" height=450 width=450>' % __counter__))


1.


2.


3.
